<div class="alert alert-info">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Привет. Меня зовут Алина Гусева, я буду делать ревью к твоей работе по предобработке данных. В дальнейшем предлагаю общаться на "ты".
    
    
Видно, что ты старался, делая проект. Все ключевые этапы проекта выполнены, в целом тебе удалось со всем справиться. Особенно хочется отметить, что соблюдена структура проекта, а код написан аккуратно.
    
Для того, чтобы и тебе и мне удобнее бы
    ло ориентироваться в комментариях, я буду помечать их разными цветами:
- красный цвет означает комментарии, которые требуют доработки. Как только ты их исправишь - проект будет принят;
- желтый цвет - то, что в следующий раз можно будет сделать по-другому. На твое усмотрение можешь доделать маленькие правки сейчас или учесть замечание для выполнения следующих проектов;
- зеленый цвет - очень элегантные и удачные решение, которые желательно использовать в дальнейшей работе.
- синий цвет - информация
    
Предлагаю работать над проектом в диалоге: если ты что-то изменяешь или вносишь какие-то коррективы - выделяй это каким-нибудь цветом и пиши об этом мне. Так мне гораздо легче будет отследить сделанные изменения.
    
Также попрошу тебя не изменять и не удалять мои комментарии, чтобы проверка твоего самостоятельного проекта прошла оперативнее.
    
 
</div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd # Подгружаем библиотеки
import math
from pymystem3 import Mystem
m = Mystem() 

data = pd.read_csv("/datasets/data.csv") # Считываем таблицу с данными, предполагаем, что иного пути хранения не может быть

data.info() # Выводим общую информацию. Замечаем, что в столбцах days_employed и total_income значений меньше чем в остальных,
# причём на одинаковое количество

# Посмотрим на данные более внимательно, выведем первые 10 строк и последние 10 строк

display(data.head(10)) # На первых десяти строках уже видны ошибки - отрицательыне значения кол-ва дней трудового стажа

display(data.tail(10)) # Также возникают вопросы касательно единиц измерения - почему пенсионер имеет 340266 дней стажа,
# неужели он работал 1000 лет? И откуда у пенсионеров доход в 160000...

# Также сомнений добавляет то, что количественные переменные - days_employed и total_income - имеют дробную часть, к тому же
# именно эти столбцы имеют пропуски. Здесь явно что-то не так. Проведём дальнейший анализ

# найдём минимальное и максимальное значения проблемных столбцов

print("Максимальный трудовой стаж",data["days_employed"].max())
print("Минимальный трудовой стаж",data["days_employed"].min())
print("Максимальный доход",data["total_income"].max())
print("Минимальный доход",data["total_income"].min())

# Видим, что минимальное значение стажа - отрицательное, но по модулю не больше модуля максимального (мне кажется, это важно)
# Исправлять будем сменой знака. Обрати внимание на пункт "*" ниже.

# Вторая проблема - максимальное число трудового стажа 401755 (предположительно дней) тогда в годах это 1100 - внушительно.
# Чтож, логично предположить, что нам нужно поделить данное число на некоторую величину для получения реальной картины.
# Я, если честно, не знаю как учитывался трудовой стаж в советском союзе в послевоенное время. Поэтому действуем таким образом:
# предполагаем, что максимальный стаж в годах - не более 60 лет, тогда 1100 надо поделить на какое-то число,
# чтобы получить примерно 60. Причём число не случайное, а специфическое - некую величину, связанную с временем. 
# На ум приходит число 24 - в одном дне 24 часа. Возможно, стаж тут считали в часах. Тогда получим, что стаж в 401755 часов - 
# - это стаж в 46 лет. Выглядит реалистично. Более подробную проверку сделаем позже.

# *
# Судя по всему, отрицательный стаж в среднем значительно меньше по модулю чем положительный, возможно, тут дело в том, что
# данный стаж указан в днях - а не в часах. Проверем данное предположение чуть позже.

# Минимальный доход = 20 тысячам, максимальный - 2.25 миллионам. Интересно познакомиться с человеком с таким доходом, но видимо
# тут всё в порядке, т.к. минимальный доход не может быть ниже прожиточного минимума (наверное).



# Проверим какие есть уникальные значения в каждом столбце с категоральными данными и посчитаем их количество.

print("Уникальные значения в столбце children \n", data["children"].value_counts())

# Чтож, 20 детей - интересный случай, но скорее всего правда в том, что ребёнка всего 2.
# Загадочный -1 ребёнок похоже на самом деле просто 1, в остальном данные пригодны

print("Уникальные значения в столбце education \n", data["education"].value_counts())

# Много повторов с разным регистром, легко исправить. Начальное - странно для человека, берущего кредит, но возможно, это 
# ребёнок войны.

print("Уникальные значения в столбце education_id \n", data["education_id"].value_counts())

# пять уровней образования, пять id - всё верно

print("Уникальные значения в столбце family_status \n", data["family_status"].value_counts())

# Здесь всё в порядке

print("Уникальные значения в столбце family_status_id \n", data["family_status_id"].value_counts())

# пять значений семейного положения, пять id - всё верно

print("Уникальные значения в столбце gender \n", data["gender"].value_counts())

# Кто такой XNA - не известно. Возможно, данные обойуться и без него. А вот то,
# что женщин в 2 раза больше чем мужчин - интересно.

print("Уникальные значения в столбце income_type \n", data["income_type"].value_counts())

# Здесь также всё выглядит нормально

print("Уникальные значения в столбце debt \n", data["debt"].value_counts())

# Значения всего 2 - 0 и 1, т.к. это булева переменная

print("Уникальные значения в столбце purpose \n", data["purpose"].value_counts())

# здесь много однотипных запросов, надо будет сгруппировать их по ключевым словам


# Ещё проверим столбец dob_years

print("Уникальные значения в столбце dob_years \n", data["dob_years"].value_counts())

# Здесь есть люди с возрастом 0 лет - возможно, для анализа нужно будет их исключить



# Вернёмся к вопросу о рабочем стаже. Найдём среднее значение стажа больше нуля и меньше нуля

print("Средний стаж", data[data["days_employed"]>0]["days_employed"].mean())

print("Средний стаж", data[data["days_employed"]<0]["days_employed"].mean())

# Значения отличаются в 155 раз, такое отличие не скомпенсирует даже перевод уменьшение в 24 раза первой величины.
# Внимательнее посмотрим на данные по самым трудящимся

display(data[data["days_employed"]>100000].head(50))
display(data[data["days_employed"]>100000].tail(50))


# Почти Все люди с таким стажем - за 50. Проверим данную теорию

hard_workers = data[data["days_employed"]>100000]
print("hardworkers", hard_workers["gender"].count())
print("young hardworkers", hard_workers[data["dob_years"]<50]["gender"].count())

# По непонятным причинам 3445 человек имеют непонятные данные по стажу.
# Проверим теорию о том, что часть данных указана в часовом формате, а часть - в нормальном.
# Для этого разделим данные на несколько категорий и посмотрим на данные

not_hard_workers = data[data["days_employed"]<=100000]

print("nothardworkers max time", not_hard_workers["days_employed"].max())

# Оказалось, что все положительные стажи - слишком завышены. А отрицательные - наоборот, выглядят нормальными (без знака)

# тогда нам нужно просто привести данные по трудовому стажу к нормальнмоу виду

# Моё предложение - не использовать данные по тем, кому младше 50 и он имеет положительный стаж (а значит завышенный),
# остальным же мы можем поделить стаж на 24, и получим результат, который может иметь место для анализа.


# Проверим пропуски - все ли строки, где нету кол-ва стажа не имеют и дохода


without_empk = data[data["days_employed"].apply(math.isnan)]

without_income = data[data["total_income"].apply(math.isnan)]

print(without_income==without_empk)

# Как видим, все пропуски встречаются там, где нету стажа и дохода одновременно. Отчасти, это логично.
# Что делать с этими данными - вопрос хороший. Посмотрим по ходу дела

# На этом обзор данных завершён, курс исправления проблем ясен.




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Максимальный трудовой стаж 401755.40047533
Минимальный трудовой стаж -18388.949900568383
Максимальный доход 2265604.028722744
Минимальный доход 20667.26379327158
Уникальные значения в столбце children 
  0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
Уникальные значения в столбце education 
 среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
Уникальные значения в столбце education_id 
 1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64
Уникальные значения в ст

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21220,0,338057.298240,68,высшее,0,женат / замужем,0,M,пенсионер,0,133344.882221,операции со своей недвижимостью
21225,0,347401.372273,60,среднее,1,женат / замужем,0,F,пенсионер,0,107546.065717,автомобили
21237,0,333140.423307,63,среднее,1,женат / замужем,0,F,пенсионер,0,98616.702311,автомобиль
21238,0,357816.497254,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,120367.978195,покупка жилья для сдачи
21240,0,344989.762566,62,среднее,1,женат / замужем,0,F,пенсионер,1,139430.605066,заняться образованием
21244,1,338558.845308,61,среднее,1,женат / замужем,0,F,пенсионер,0,96218.943434,свой автомобиль
21249,0,333693.916238,67,среднее,1,Не женат / не замужем,4,F,пенсионер,0,109336.689515,строительство собственной недвижимости
21253,0,340940.073979,55,среднее,1,гражданский брак,1,F,пенсионер,0,119509.306905,покупка своего жилья
21254,1,385304.641551,61,среднее,1,женат / замужем,0,F,пенсионер,0,138045.369909,строительство недвижимости
21257,0,379985.925569,72,среднее,1,женат / замужем,0,F,пенсионер,0,60867.499554,получение дополнительного образования


hardworkers 3445
young hardworkers 182
nothardworkers max time -24.14163324048118
       children  days_employed  dob_years  education  education_id  \
12         True          False       True       True          True   
26         True          False       True       True          True   
29         True          False       True       True          True   
41         True          False       True       True          True   
55         True          False       True       True          True   
...         ...            ...        ...        ...           ...   
21489      True          False       True       True          True   
21495      True          False       True       True          True   
21497      True          False       True       True          True   
21502      True          False       True       True          True   
21510      True          False       True       True          True   

       family_status  family_status_id  gender  income_type  debt  \
12      

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:115: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


**Вывод**

В ходе исследования данных были выявлены следующие проблемы:

1) пропущенные значения в столбцах days_employed и total_income (они идут синхронно) 

2) ошибочные данные, которые можно исправить, в столбцах children, dob_years

3) повторяюшиеся данные в столбце education - их нужно привести к пяти типам

4) повторяюшиеся данные в столбце purpose - их нужно лемматизировать и сгруппировать

5) данные, которые исправить нельзя (но, возможно, и не надо): одно значение в столбце gender, 182 значения в столбце days_employed, которые лучше будет убрать (те, что относятся к условной группе young hardworkers) и 101 ноль-летний в столбце dob_years. Это чуть больше 1%, так что ничего страшного не случиться, если эти данные удалить. Исправить их не представляется возможным, по крайней мере пока.

Можно сказать, большинство проблем с данными решаются элементарно. Вопрос состоит в загадке огромного трудового стажа некоторых людей. Мы предположили, что эти данные записаны в часах (т.к. для людей за 50 такой стаж выглядит нормальным). Также осталась непонятна причина дробной части значений столбцов days_employed и total_income.

Касательно столбца total_income - было сделано предположение, что с данным столбцов всё в порядке, т.к. не у всех людей довольно большой доход (как у пенсионеров в 150к рублей)

Следующим шагом будет устранение текущих проблем с данными и их последующая оценка.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Молодец, хороший тон использовать метод info() для получения общей информации из таблицы. Для более детального изучения можно воспользоваться методом describe(). Ты очень ответственно подошел у изучению предоставленных данных. 
Действительно, в таблице много пропусков и некорректных значений, все как в реальной жизни. 
    
 
</div>

<div class="alert alert-info">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Для того, чтобы и тебе и ревьюерам было удобнее проверять проекты, советую тебе отделять комментарии от кода. Вообще, строчные комментарии(начинающиеся со #) предназначены для кратких заметок, чтобы тебе проще в дальнейшем было ориентирвоаться в коде. Если же ты делаешь какие-то выводы, предположения, глядя на полученный результат, то лучше такие комментарии записывать в отдельную ячейку, используя формат markdown.   
 
</div>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# Начнём исправление ошибок в данных с обработки пропусков.
# Пропуски могли возникнутьв связи с нежеланием клиента предоставлять свои данные о доходах,
# что странно, ведь так могут и не дать кредит. Возможно, клиент не мог разглашать информацию, но таких случаев
# явно не 2000 из 20 тысяч, поэтому скорее всего здесь сыграл просто человеческий фактор. 
# Пропуски в столбце стажа работы не интересуют нас для исследования, поэтому на них можно закрыть глаза,
# однако, в этих же строках отсутствует значение дохода - а это один из факторов, которые нас интересуют. 
# Благодаря тому, что у нас есть столбец income_type мы знаем, что данные клиенты работают/пенсионеры, 
# на основании чего мы можем рассчитать средний доход для каждой категории, и заменить пустоту на требуемое значение.
# Столбец days_employed смысла трогать нету, т.к. он нам не пригодится (по крайней мере сейчас нету причин, почему он нам нужен)
# Нас интересуют основные 4 вида заработка, а всего их 8 - последние 4 в сумме имеют 6 человек, поэтому средюю запрлату
# для них считать не будет, а если останется пропуск - то можно спокойно удалить.

# рассчитаем средний доход для каждого типа клиента (за исключеним 4 маловстречающихся)

mean_emp = data[data["income_type"]=="сотрудник"]["total_income"].mean() # для сотрудников
mean_comp = data[data["income_type"]=="компаньон"]["total_income"].mean()# для компаньонов
mean_ret = data[data["income_type"]=="пенсионер"]["total_income"].mean()# для пенсионеров
mean_gov = data[data["income_type"]=="госслужащий"]["total_income"].mean() # для госслужащих

print(mean_emp, mean_comp, mean_ret, mean_gov) # убеждаемся в том, что мы получили данные

# Теперь пора заполнить пропуски

without_income[without_income["income_type"]=="сотрудник"]= without_income[without_income["income_type"]=="сотрудник"].fillna(mean_emp)
without_income[without_income["income_type"]=="компаньон"]= without_income[without_income["income_type"]=="компаньон"].fillna(mean_comp)
without_income[without_income["income_type"]=="пенсионер"]= without_income[without_income["income_type"]=="пенсионер"].fillna(mean_ret)
without_income[without_income["income_type"]=="госслужащий"]= without_income[without_income["income_type"]=="госслужащий"].fillna(mean_gov)

data[data["total_income"].apply(math.isnan)] = without_income

data.info() # убеждаемся в том, что пустых значений в total_income не осталось. Всё же один пропуск есть. Удалим его


data = data.dropna().reset_index(drop = True)

data.info() # Теперь все пропуски заполнены, пусть и столбец days_employed стал нести лишнюю информацию

161380.26048788553 202417.4614617771 137127.4656901654 170898.30992266268
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21524 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21524 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

**Вывод**

Все пропуски в файле были успешно устранены, зарпалата была заполнена средними значениями зарплат для соответствующих категорий работников. При таком заполнении пострадала колокна с опытом работы, но нас это не сильно волнует, так как она не будет использоваться в дальнейшем. Теперь с этими данными можно делать другие преобразования и операции, не боясь ошибок с пустотами. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Ты верно заметил, что пропуски есть только в двух столбцах, причем они симметричны. Поскольку для дальнейшего исследования нам не очень важен стаж, пропуски в этом столбце можно оставить как есть. Но обязательно сообщить разработчикам базы данных о подобной проблеме. А вот с столбцом с зарплатой немного интереснее. Действительно, уровень дохода зависит от множества факторов. В данном случае можно предположить, что на уровень дохода влияет тип занатости или образование. То есть можно группировать заемщиков по одонму из этих параметров и в каждой группе искать среднее/ медианной значение. Ты делаешь все правильно, но не самым оптимальным способом. На будущее есть очень полезный метод, который помогает произвести вышеописанные подсчеты в одну строчку. Метод <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html'>transform()</a>
 
</div>

### Замена типа данных

In [3]:
# Теперь решим вопрос с данными в ошибочном формате. В первую очередь сделаем доход и стаж целыми числами.
# Замену сделали через метод astype, так как отбросить дробную часть без округления не проблема в нашем случае

data["days_employed"] = data["days_employed"].astype("int")

data["total_income"] = data["total_income"].astype("int")

data.info() # Убеждаемся в том, что все данные преобразовались так, как мы хотим.

# Теперь исправим другие проблемные данные в children

data["children"] = data["children"].replace(20,2)

print(data[data["children"]==20]["children"].count()) # Видим, что одну замену сделали верно, делаем следующую

data["children"] = data["children"].replace(-1,1)

print(data[data["children"]==-1]["children"].count()) # Видим, что и тут заменили верно

# Проблему с данными в столбце days_employed пока трогать не будем, так как он нам не нужен

# Вопрос с удалением данных, которые нельзя исправить (см. пункт 5 в выводе по первой части) можно пока не решать в силу того,
# что они нам не нужны для ответа на вопросы, которые поставил перед нами банк

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
0
0


**Вывод**

Мы привели данные, которые будем использовать далее, в порядок - а именно перевели значения дохода в целые числа, а также сделали количество детей "нормальным". Теперь можно взяться за решение других проблем с данными. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Отлично, теперь все столбцы имеют корректный формат данных. Кстати, чтобы избавиться от отриуательных значений, можно использовать метод abs(), который возвращает модуль числа
</div>

### Обработка дубликатов

In [4]:
# Разберёмся с дупликатами некоторых значений, таких как в столбце education.
# Благодаря наличию id образования, мы можем однозначно присвоить в каждую ячейку нужное значение.
# Дубликаты могли появиться в следствии того, что каждый сотрудник банка по своему забивает данные

data.loc[data["education_id"]==0,"education"]="высшее"
data.loc[data["education_id"]==1,"education"]="среднее"
data.loc[data["education_id"]==2,"education"]="неоконченное высшее"
data.loc[data["education_id"]==3,"education"]="начальное"
data.loc[data["education_id"]==4,"education"]="учёная степень"

print("Уникальные значения в столбце education \n", data["education"].value_counts()) # проверямем правильность замены


Уникальные значения в столбце education 
 среднее                15233
высшее                  5259
неоконченное высшее      744
начальное                282
учёная степень             6
Name: education, dtype: int64


**Вывод**

Теперь мы исправили ещё одну проблему с данными, а именно разное написание одной и той же информации. Осталось привести всего пару других столбцов в нормальный вид и можно начинать анализ.

<div class="alert alert-warning">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Да, в данном случае в столбце с образованием встречалось различное написание одного и того же слова. Можно делать "в лоб", а можно просто все слова привести к ниженму регистру. Для этого к столбцу применить метод str.lower(). И еще, здесь важно посмотреть количество дубликатов во всей таблице. Может быть такое, что информация задвоилась, то есть один и тот же заемщик записан два раза. И такие клиенты будут влиять на дальнейший результат. Лушче всего проверить наличие повторов методом duplicated(), а если такие встречаются --- удалить их методом drop_duplicates().
</div>

### Лемматизация

In [5]:
# Столбцы с детьми и уровнем дохода мы исправили, данные по семейному положению были в порядке. Теперь осталось заняться
# Столбцом purpose. Здесь требуется использовать лемматизацию для выявления схожих запросов и их объединения.
# Напомним себе ещё раз какие цели заявляли клиенты. 

print("Уникальные значения в столбце purpose \n", data["purpose"].value_counts())

# Можно выделить такие цели как: 1) свадьба 2) операции с личной недвижимостью (жильё) 3) операции с коммерческой недвижимостью
# 4) ремонт 5) покупка автомобиля 6) образование
# Если не сказано, что недвижимость коммерческая, то считаем что она личная. Ремонт может конечно относиться к автомобилю, но
# в нашших данных нету такого пункта, поэтому не будем учитывать такой вариант.

# подготовим данные для проведения замены

words = 'свадьба-коммерческая-ремонт-автомобиль-образование-недвижимость-жильё' # создаём слова для лемматизации
# здесь важен порядок, т.к. если у нас есть ремонт жилья, то сначала должно поменять название на ремонт, иначе операция уйдёт
# в жильё. Аналогично с коммерческой недвижимостью.

lemmas = m.lemmatize(words) # переводим слова в леммы
print(lemmas) # выводим леммы на экран

# теперь займёмся непосредственно заменой


for lem in lemmas: # Если честно, я не придумал как сделать лучше, поэтому сделал через такой цикл, считает немного долго!
    for index in range(data.shape[0]): 
        if lem in m.lemmatize(data.loc[index,"purpose"]) and lem != '\n':
            data.loc[index,"purpose"] = lem
            
# Заменим жильё на недвижимость, так как речь идёт об одном и том же

data["purpose"] = data["purpose"].replace("жилье","недвижимость")
            
print("Уникальные значения в столбце purpose \n", data["purpose"].value_counts()) # Как видим, теперь все цели ясны -
# - их всего 6. Теперь работать с этими данными намного проще.

Уникальные значения в столбце purpose 
 свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                606
на покупку своего автомобиля            

**Вывод**

Теперь, когда мы разобрались со столбцом purpose, сгруппировать данные намного проще. При группировки мы пренебрегли небольшими нюансами, такими как покупка жилья для себя, какая-то специфическая операция с жильём (возможно, обмен с доплатой) или же покупка для сдачи. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Отлично, лемматизация прошла успешно, удалось выделить основные цели получения кредита.
</div>

### Категоризация данных

In [6]:
# Для удобства работы с данными разделим их на несколько категорий.
# Будем брать для каждого необходимого нам вопроса требуемые столбцы из главного файле, но предварительно дополним его
# новыми столбцами, чтобы было проще ориентироваться в информации. 

# Рассмотрим первый вопрос: Есть ли зависимость между наличием детей и возвратом кредита в срок?
# Для этого нам потребуются столбцы children, debt. Данные по кол-ву детей мы уже исправляли раньше. 

# Рассмотрим второй вопрос: Есть ли зависимость между семейным положением и возвратом кредита в срок?
# Для ответа нам потребуются столбцы family_status_id и debt. Столбец с расшифровкой id тащить сюда не будем

# Рассмотрим третий вопрос: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
# Для ответа нам потребуются столбцы total_income и debt. С первым столбцом мы уже работали, так что там данные корректные.
# Здесь потребуется разделение на условные категории по зарплате, что мы сделаем чуть позже 

# Рассмотрим чертвёртый вопрос: Как разные цели кредита влияют на его возврат в срок?
# Для ответа нам потребуются столбцы purpose и debt. Первый столбец мы уже подготовили, так что работать с ним будет удобно.


# Создаём новые дата-фреймы для удобной работы

children_q = data[['children', 'debt']]

family_q = data[['family_status_id', 'debt']]

income_q = data[['total_income', 'debt']]

purpose_q = data[['purpose', 'debt']]

display(children_q) # Убеждаемся, что всё скопировали корректно

display(family_q)

display(income_q)

display(purpose_q)

# Теперь можно приступать к ответам на сами вопросы

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0
...,...,...
21519,1,0
21520,0,0
21521,1,1
21522,3,1


,family_status_id,debt
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
21519,1,0
21520,0,0
21521,1,1
21522,0,1


,total_income,debt
0,253875,0
1,112080,0
2,145885,0
3,267628,0
4,158616,0
...,...,...
21519,224791,0
21520,155999,0
21521,89672,1
21522,244093,1


,purpose,debt
0,недвижимость,0
1,автомобиль,0
2,недвижимость,0
3,образование,0
4,свадьба,0
...,...,...
21519,недвижимость,0
21520,автомобиль,0
21521,недвижимость,1
21522,автомобиль,1


**Вывод**

Теперь, когда мы подготовили удобные датафреймы для работы, мы можем начать основную часть работы. Для этого у нас готовы 4 датафрейма только с необходимой информацией.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# Чтобы ответить на этот вопрос мы будем использовать датафрейм children_q
# найдём среднее значение отдачи в процентах в зависимости от кол-ва детей
# Объеденим данные для семей с тремя, четырься и пятью детьми в одно значение т.к. выборка становиться меньше при повышении
# кол-ва детей

mean = []

for i in range(3):
    mean.append(round(children_q[children_q["children"]==i]["debt"].mean()*100,2))
    
child_3 = round(children_q[children_q["children"]==3]["debt"].mean(),2)
child_4 = round(children_q[children_q["children"]==4]["debt"].mean(),2)  
child_5 = round(children_q[children_q["children"]==5]["debt"].mean(),2)  
child_mean = (child_3+child_4+child_5)/3
mean.append(round(child_mean*100,2))
print(mean)

# на основании полученных данных можно высказать определённую теорию

[7.51, 9.15, 9.48, 6.0]


**Вывод**

Судя по данным, клиенты без детей уплачивают кредиты в срок чаще, чем с одним или двумя детьми. Такие результаты вполне логичны, так как отсутствие детей позволяет больше концентрироваться на заработке денег, к тому же на жизнь получается меньше расходов. Но также данные говорят о том, что клиенты с 3 и более детьми в среднем выплачивают кредиты в срок чаще, чем все остальные. Это теория может стыковаться с первой теорией в том случае, если предположить, что банки изначально выдают кредиты семьям с 3 и более детьми в том случае, если считают их платёжеспособными, другими словами, такая семмья должна соответствовать более строгим критериям банка, чтобы получить кредит (и скорее всего это правда так). Тогда теория о том, что количество детей в семье влияет на её платёжеспособность по кредиту, верна. Однако данную теорию хорошо бы уточнить у банковского отдела кредитования, так как без информации о количестве отказов по кредитам для разных людей не получится подтвердить данное предположение. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Верно, замещики без детей самые ответственные)
    
Касаемо кода --- его лучше оптимизировать. Я не вижу смысла создавать отдельные датафреймы, это только засоряет проект. Цикл тоже довольно долго отрабатывает. Здесь очень удобно воспользоваться методом pivot_table(), то есть сгруппировать данные по нужному признаку и вывести процент должников в каждой группе.
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# Чтобы ответить на этот вопрос мы будем использовать датафрейм family_q 
# найдём среднее значение отдачи в процентах в зависимости от семейного положения



for i in family_q['family_status_id'].unique():
    temp = round(family_q[family_q["family_status_id"]==i]["debt"].mean()*100,2)
    if i == 0:
        print("Клиенты в браке имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == 1:
        print("Клиенты в гражданском браке имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == 2:
        print("Клиенты вдовцы/вдовы имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == 3:
        print("Клиенты в разводе имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == 4:
        print("Неженатые клиенты имеют задолжности по кредиту в", temp ,"процентах случаев")

# на основании полученных данных можно высказать определённую теорию
 

Клиенты в браке имеют задолжности по кредиту в 7.52 процентах случаев
Клиенты в гражданском браке имеют задолжности по кредиту в 9.29 процентах случаев
Клиенты вдовцы/вдовы имеют задолжности по кредиту в 6.56 процентах случаев
Клиенты в разводе имеют задолжности по кредиту в 7.11 процентах случаев
Неженатые клиенты имеют задолжности по кредиту в 9.74 процентах случаев


**Вывод**

Выводы, полученные в результате анализа данных говорят о том, что люди, когда-либо вступавшие в брак платят кредит в срок чаще, чем те, которые не вступали (официально). И это логично, потому что большинство людей вступают в брак в более сознательном возрасте, и с более серьёзными планами на жизнь. Поэтому таких людей можно назвать более стабильными. Касательно вдов и вдовцов - дело в том, что такие случае чаще случаются в пожилом возрасте, чем в молодом, и, с большой вероятностью, люди постарше выплачивают кредиты лучше тех, кто моложе. Конечно, это тоже неплохо бы проверить, но такой задачи пока не стояло. По этим данным однозначно можно сказать, что семейное положение влияет на возврат кредита в срок.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.
    
</div>


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
# Чтобы ответить на этот вопрос мы будем использовать датафрейм income_q 
# найдём среднее значение отдачи в процентах в зависимости от уровня дохода
# для этого сделаем условное разбиение клиентов на несколько групп по их уровню дохода
# от 20 до 70 тысяч - бедные, от 70 до 140 - средний класс, от 140 до 250 - продвинутый класс
# от 250 до 500 - богатые, выше 500 - высший класс. (это условно-субъектное разделение)

# Теперь напишем функцию для определения класса

def defineclass(income):
    if income< 70000:
        return "бедный"
    if income< 140000:
        return "средний"
    if income< 250000:
        return "продвинутый"
    if income< 500000:
        return "богатый"
    return "высший класс"


income_q["class"]=income_q["total_income"].apply(defineclass) 

print(income_q) # Проверяем адекватность данных

print(income_q["class"].value_counts()) 

print()

for i in income_q['class'].unique():
    temp = round(income_q[income_q["class"]==i]["debt"].mean()*100,2)
    if i == "бедный":
        print("Бедные клиенты имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "средний":
        print("Клиенты со средним достатком имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "продвинутый":
        print("Клиенты продвинутого класса имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "богатый":
        print("Богатые клиенты имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "высший класс":
        print("Клиенты высшего класса имеют задолжности по кредиту в", temp ,"процентах случаев")
        
# на основании полученных данных можно высказать определённую теорию


       total_income  debt        class
0            253875     0      богатый
1            112080     0      средний
2            145885     0  продвинутый
3            267628     0      богатый
4            158616     0  продвинутый
...             ...   ...          ...
21519        224791     0  продвинутый
21520        155999     0  продвинутый
21521         89672     1      средний
21522        244093     1  продвинутый
21523         82047     0      средний

[21524 rows x 3 columns]
продвинутый     9194
средний         8044
богатый         2590
бедный          1474
высший класс     222
Name: class, dtype: int64

Богатые клиенты имеют задолжности по кредиту в 6.95 процентах случаев
Клиенты со средним достатком имеют задолжности по кредиту в 8.45 процентах случаев
Клиенты продвинутого класса имеют задолжности по кредиту в 8.33 процентах случаев
Бедные клиенты имеют задолжности по кредиту в 6.85 процентах случаев
Клиенты высшего класса имеют задолжности по кредиту в 6.31 процентах с

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


**Вывод**

Мы можем наблядать, что доход клиентов несильно влияет на способность выплачивать кредиты. Точнее, он влияет меньше, чем остальные параметры. Основная тенденция такова: больше всего не выплачивают клиенты среднего или продвинутого класса, т.е. с зарплатами от 70 до 250 тысяч. Далее с повышением зарплаты платёжеспособность растёт, что логично. Но почему бедные клиенты платят также хорошо, как и богатые? Возможно, к клиентам с низким доходом более строгие требования для выдачи, поэтому они и платят чаще В срок. Конечно, данные говорят что данный фактор также влияет на платёжеспособность, однако немного меньше остальных, и намного меньше, чем ожидалось изначально.

<div class="alert alert-info">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
В целом, все так, сейчас ты сделал условно-субъективное деление. А как быть если данных осень много и нельзя с первого взгляда их поровну поделить на группы? Для этого есть метод <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html'>quantile()</a>, который позволяет раздлеить данные на статистически равные группы. Или же метод qcut(), где в качества параметра необходимо ввести число групп.
    
</div>


- Как разные цели кредита влияют на его возврат в срок?

In [10]:
# Чтобы ответить на этот вопрос мы будем использовать датафрейм purpose_q
# найдём среднее значение отдачи в процентах для каждой цели кредита

for i in purpose_q['purpose'].unique():
    temp = round(purpose_q[purpose_q['purpose']==i]["debt"].mean()*100,2)
    if i == "недвижимость":
        print("Клиенты с целью покупки недвижимости имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "автомобиль":
        print("Клиенты с целью покупки автомобиля задолжности по кредиту в", temp ,"процентах случаев")
    if i == "образование":
        print("Клиенты с целью образования имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "свадьба":
        print("Клиенты с целью свадьбы имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "коммерческий":
        print("Клиенты с целью покупки коммерческой недвижимости имеют задолжности по кредиту в", temp ,"процентах случаев")
    if i == "ремонт":
        print("Клиенты с целью ремонта имеют задолжности по кредиту в", temp ,"процентах случаев")

        
# на основании полученных данных можно высказать определённую теорию      

# проведём дополнительное исследование для выяснения влияния возраста на способность уплаты кредита вовремя
def defineclass(age):
    if age< 30:
        return "юный"
    if age< 40:
        return "молодой"
    if age< 50:
        return "взрослый"
    if age< 60:
        return "пожилой"
    return "пенсионер"

import numpy # отчищаем данные от нулевого возраста для последнего ислледования
data["dob_years"] = data["dob_years"].replace(0,numpy.nan)
data = data.dropna().reset_index(drop=True)
data.info()
data["dob_years"] = data["dob_years"].astype("int")

data["age_class"]=data["dob_years"].apply(defineclass) 

for i in data["age_class"].unique():
    print(i)
    print(round(data[data["age_class"]==i]["debt"].mean()*100,2))
    
# можем наблюдать ожидаемый результат, возраст напрямую влияет на способность уплаты кредита вовремя, чем старше человек -
# - тем выше шанс, что он уплатит кредит вовремя. Это делает последующие выводы верными.

Клиенты с целью покупки недвижимости имеют задолжности по кредиту в 7.27 процентах случаев
Клиенты с целью покупки автомобиля задолжности по кредиту в 9.34 процентах случаев
Клиенты с целью образования имеют задолжности по кредиту в 9.2 процентах случаев
Клиенты с целью свадьбы имеют задолжности по кредиту в 7.92 процентах случаев
Клиенты с целью покупки коммерческой недвижимости имеют задолжности по кредиту в 7.53 процентах случаев
Клиенты с целью ремонта имеют задолжности по кредиту в 5.72 процентах случаев
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21423 entries, 0 to 21422
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null float64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
d

**Вывод**

Полученные данные говорят о том, что наиболее кредитовозвратная цель - это ремонт, возможно потому, что суммы на ремонт не самые большие, по сравнению с другими целями. Самые проблемные - это автомобили и образование - с образованием понятно, потому что чаще всего кредит на него берут ещё молодые люди, у которых не всегда есть стабильный источник дохода. С автомобилем таже история, обычно в кредит берут молодые люди, т.к. у более пожилых людей обычно есть сбережения на такие небольшие траты. С ремонтом дело обстоит по другому. Ремонт обычно происходит либо в новой квартире, либо в той, где люди жили долгое время с собераются что-либо поменять, и опять таки же, это обычно совершают более взрослые люди. Вывод такой - цель кредита влияет на возможность возврата денег в срок. 

## Шаг 4. Общий вывод

На основании всех полученных данных можно сделать вывод о том, что наибольшее влияние из 4 предполагаемых факторов оказывает семейное положение и цель кредита, а они, в свою очередь, во многом связаны с возрастом, который напрямую, и сильнее всех, влияет на способность возврата кредита. Факторы количества детей и зарплаты влияют немного меньше, но всё же вносят свою лепту. Поэтому можно утверждать, что все проверенные факторы имеют значимость при анализе потенциального потребителя кредита. Однако для более точного вывода по факторам зарплаты и количества детей требуется дополнительная информация от банка касательно процентного соотношения одобренных кредитов к отказам для данных случаев. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Поздравляю с первым успешным  проектом!

- Ты хорошо делаешь выводы и анализируешь результаты.
    
- Код написан аккуратно и качественно, видно, что есть какой-то определенный бэкграунд.

- Соблюдена структура проекта, было приятно проверять:)
    
- Все написано четко и по делу

 

Желаю дальнейших успехов!


</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.